# Image Classification of ATLAS Calorimeter Topo-Clusters (Jan)

This is a stripped-down version of Max's re-write, so I have removed *some* functionality.

In [1]:
# If true, force re-training even if a model already exists. Existing model will be lost!
overwriteModel = False

# If true, continue training and try to train to the last specified epoch.
# If EarlyStopping was used, this may result in trying to further train a "finished" network.
finishTraining = False

# If no file extension, uses native TensorFlow format (.tf).
# If 'h5', uses HDF5. HDF5 does not work for custom layers/classes! (unless you design them a certain way)
file_extension = '.h5'
if(file_extension != '' and '.' not in file_extension):
    file_extension = '.' + file_extension

## Quick Navigation:
- [Simple feed-forward Neural Network](#Simple-feed-forward-Neural-Network)
- [Combination Network](#Combination-Network)
- [Convolutional Neural Networks](#Convolutional-Neural-Networks)
    - [Single-calo-layer CNN](#Single-calo-layer-CNN's)
    -[Combination Network (CNN's)](#Combination-Network:-Take-2)
    - [Complex CNN's](#Convolutional-Neural-Networks:-More-complicated-architectures)
        - [3 EMB layers, separate](#3-EMB-layers,-separate)
        - [All layers, separate](#All-layers,-separate)
        - [All layers, merged](#All-layers,-merged)
        - [EMB merged + TileBar merged](#EMB-merged-+-TileBar-merged)
        - [EMB merged + TileBar depth](#EMB-merged-+-TileBar-depth)
- [ResNet](#ResNet)
- [Combination Network (ResNet + simple NN)](#Combination-Network:-Take-3)

- [Summary Plots](#Summary-Plots)

In [2]:
#import libraries and some constants
import os, sys, json, pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ROOT as rt # I will use this for some plotting
import uproot as ur

path_prefix = os.getcwd() + '/../'
plotpath = path_prefix+'classifier/Plots/'
modelpath = path_prefix+'classifier/Models/'
# %config InlineBackend.figure_format = 'svg'

if(path_prefix not in sys.path): sys.path.append(path_prefix)
from util import resolution_util as ru
from util import plot_util as pu
from util import ml_util as mu
from util import qol_util as qu

# Custom tensorflow.keras callbacks
from util.keras.callbacks import GetCallbacks

# Classification-specific utils
from util.classification import training_util as ctu
from util.classification import plot_util as cpu
from util.classification import data_util as cdu

# metadata
layers = ["EMB1", "EMB2", "EMB3", "TileBar0", "TileBar1", "TileBar2"]
cell_size_phi = [0.098, 0.0245, 0.0245, 0.1, 0.1, 0.1]
cell_size_eta = [0.0031, 0.025, 0.05, 0.1, 0.1, 0.2]
len_phi = [4, 16, 16, 4, 4, 4]
len_eta = [128, 16, 8, 4, 4, 2]
cell_shapes = {layers[i]:(len_eta[i],len_phi[i]) for i in range(len(layers))}

Welcome to JupyROOT 6.22/02


In [3]:
# fancy display names for each pion type
pi_latex = {
    'pi0': '\(\pi^{0}\)',
    'piplus': '\(\pi^{+}\)',
    'piminus': '\(\pi^{-}\)',
}

In [4]:
# Plotting settings

# plotting mode
mode = 'dark'

# xkcd -- turn this on for fun-looking (but marginally less useful) plots
use_xkcd = False
if(use_xkcd):
    mode = 'light'
    plt.xkcd(scale=.75,length=100,randomness=1)
    
# plotting style -- manages our color palette and object colors
plotstyle = qu.PlotStyle(mode)
    
# some matplotlib-specific stuff
params = {'legend.fontsize': 13,
          'axes.labelsize': 18}
plt.rcParams.update(params)

Now, we will import our data from the `ROOT` files into a `pandas` DataFrame. The first cell takes care of scalars, and the second takes care of vectors.

In [5]:
# import pi+- vs. pi0 images
source = 'pion' # also try 'jet'

if(source == 'pion'):
    inputpath = path_prefix+'data/pion/'
    rootfiles = ["pi0", "piplus", "piminus"]
    branches = ['runNumber', 'eventNumber', 'truthE', 'truthPt', 'truthEta', 'truthPhi', 'clusterIndex', 'nCluster', 'clusterE', 'clusterECalib', 'clusterPt', 'clusterEta', 'clusterPhi', 'cluster_nCells', 'cluster_sumCellE', 'cluster_ENG_CALIB_TOT', 'cluster_ENG_CALIB_OUT_T', 'cluster_ENG_CALIB_DEAD_TOT', 'cluster_EM_PROBABILITY', 'cluster_HAD_WEIGHT', 'cluster_OOC_WEIGHT', 'cluster_DM_WEIGHT', 'cluster_CENTER_MAG', 'cluster_FIRST_ENG_DENS', 'cluster_cell_dR_min', 'cluster_cell_dR_max', 'cluster_cell_dEta_min', 'cluster_cell_dEta_max', 'cluster_cell_dPhi_min', 'cluster_cell_dPhi_max', 'cluster_cell_centerCellEta', 'cluster_cell_centerCellPhi', 'cluster_cell_centerCellLayer', 'cluster_cellE_norm']
elif(source == 'jet'):
    inputpath = path_prefix+'jets/training/'
    rootfiles = ["pi0", "piplus"]
    branches = ['runNumber', 'eventNumber', 'truthE', 'truthPt', 'truthEta', 'truthPhi', 'clusterIndex', 'nCluster', 'clusterE', 'clusterECalib', 'clusterPt', 'clusterEta', 'clusterPhi', 'cluster_nCells', 'cluster_ENG_CALIB_TOT']
else:
    assert(False)

trees = {
    rfile : ur.open(inputpath+rfile+".root")['ClusterTree']
    for rfile in rootfiles
}
pdata = {
    ifile : itree.pandas.df(branches, flatten=False)
    for ifile, itree in trees.items()
}

total = 0
for key in rootfiles:
    total += len(pdata[key])

for key in rootfiles:
    n = len(pdata[key])
    print("Number of {a:<7} events: {b:>10}\t({c:.1f}%)".format(a=key, b = n, c = 100. * n / total))
print("Total: {}".format(total))

Number of pi0     events:     264167	(23.2%)
Number of piplus  events:     436949	(38.4%)
Number of piminus events:     436802	(38.4%)
Total: 1137918


The number of events for each category may be quite different -- ultimately we want to train our classifier on a "balanced" dataset, where we have equal numbers of entries from each category.

We're training our network to classify between $\pi^\pm$ and $\pi^0$ events. Thus, we should ultimately merge our $\pi^+$ and $\pi^-$ data.

Thus, we will first generate selected indices for all categories, such that the total number of events from each category is equal, and *then* we will merge things.

Note that as we're dealing with DataFrames (`pdata`) and uproot trees (`trees`, whose contents get loaded into `pcells`), we have to be careful that when we merge data, we do it the same way for both sets of objects. Otherwise we might scramble our $\pi^\pm$ data.

In [6]:
n_indices = {}
n_max = int(np.min(np.array([len(pdata[key]) for key in trees.keys()])))
rng = np.random.default_rng()

# If we have a piminus key, assume the dataset are piplus, piminus, pi0
if('piminus' in trees.keys()):
    n_indices['piplus']  = int(np.ceil((n_max / 2)))
    n_indices['piminus'] = int(np.floor((n_max / 2)))
    n_indices['pi0']     = n_max
    
# Otherwise, assume we already have piplus (or piplus + piminus) and pi0, no merging needed
else: n_indices = {key:n_max for key in trees.keys}
indices = {key:rng.choice(len(pdata[key]), n_indices[key], replace=False) for key in trees.keys()}

# Make a boolean array version of our indices, since pandas is weird and doesn't handle non-bool indices?
bool_indices = {}
for key in pdata.keys():
    bool_indices[key] = np.full(len(pdata[key]), False)
    bool_indices[key][indices[key]] = True

# Apply the (bool) indices to pdata
for key in trees.keys():
    pdata[key] = pdata[key][bool_indices[key]]

# prepare pcells -- immediately apply our selected indices
pcells = {
    ifile : {
        layer : mu.setupCells(itree, layer, indices = indices[ifile])
        for layer in layers
    }
    for ifile, itree in trees.items()
}

In [7]:
# Now with the data extracted from the trees into pcells, we merge pdata and pcells as needed.
# Note the order in which we concatenate things: piplus -> piplus + piminus.
if('piminus' in trees.keys()):
    print('Merging piplus and piminus.')
    
    # merge pdata
    pdata['piplus'] = pdata['piplus'].append(pdata['piminus'])
    del pdata['piminus']
    
    # merge contents of pcells
    for layer in layers:
        pcells['piplus'][layer] = np.row_stack((pcells['piplus'][layer],pcells['piminus'][layer]))
    del pcells['piminus']

Merging piplus and piminus.


In [8]:
pdata_merged, pcells_merged, plabels = cdu.DataPrep(pdata, pcells, layers, trainfrac=0.7)

We'll train the network on $\pi^+$ and $\pi^0$ events.

### Plot a few example images.

These are the images that we will use to train our network (together with a few other variables).

In [ ]:
cpu.ImagePlot(
    pcells,
    cluster=100,
    layers=layers,
    cell_shapes=cell_shapes,
    plotpath=plotpath,
    filename='calo_images.png',
    plotstyle=plotstyle
)

In [ ]:
scaled_shape = (16,16)
cpu.ImagePlot(
    pcells,
    cluster=100,
    scaled_shape=scaled_shape,
    layers=layers,
    cell_shapes=cell_shapes,
    plotpath=plotpath,
    filename='calo_images_{}x{}.png'.format(*scaled_shape),
    plotstyle=plotstyle
)

### Plot a few histograms.

These are a bit uglier than the `matplotlib` ones Max made, but it's perhaps even easier to see any differences between $\pi^\pm$ and $\pi^0$.

In [ ]:
rt.gStyle.SetOptStat(0)

plotstyle.SetStyle()

# For storing histograms and legends, to prevent overwriting. (TODO: Probably better ways to do this in PyROOT)
histos = []
legends = []

qtys = ['cluster_nCells', 'clusterE', 'clusterEta', 'clusterPhi', 'cluster_EM_PROBABILITY', 'cluster_sumCellE']
qty_labels = ['Cells/Cluster', 'Cluster Energy [GeV]', 'Cluster #eta', 'Cluster #phi', 'Cluster EMProb', 'Cluster SumCellE']
qty_ranges = [(0,500), (50,200), (-0.8,0.8), (-4.,4.), (0.,1.), (0.,2500.)]

if(source == 'jet'):
    qtys = ['cluster_nCells', 'clusterE', 'clusterEta', 'clusterPhi']
    qty_labels = ['Cells/Cluster', 'Cluster Energy [GeV]', 'Cluster #eta', 'Cluster #phi']
    qty_ranges = [(0,300), (0,100), (-0.8,0.8), (-4.,4.)]

# Set up a canvas.
plot_size = 500
nx = int(np.ceil(len(qtys) / 2))
ny = 2
n_pad = nx * ny
canvas = rt.TCanvas('cluster_hists','c1',plot_size * nx,plot_size * ny)
canvas.Divide(nx,ny)

colors = {'piplus':rt.kRed,'pi0':rt.kBlue}
styles = {'piplus':3440, 'pi0':3404}

n_bins=20
for i, (qty, label, rng) in enumerate(zip(qtys, qty_labels, qty_ranges)):
    canvas.cd(i+1)
    leg = rt.TLegend(0.7,0.8,0.9,0.9)
    for ptype, p in pdata.items():
        hist = rt.TH1F('h_'+str(ptype)+'_'+str(qty),'',n_bins,rng[0],rng[1])
        for entry in p[qty]: hist.Fill(entry)
        integral = hist.Integral()
        if(integral != 0): hist.Scale(1./hist.Integral())
        hist.SetLineColor(colors[ptype])
        hist.SetLineWidth(2)
        hist.SetFillColorAlpha(colors[ptype],0.5)
        hist.SetFillStyle(styles[ptype])
        hist.Draw('HIST SAME')
        hist.GetXaxis().SetTitle(label)
        hist.GetYaxis().SetTitle('Normalised events')
        hist.SetMaximum(1.5 * hist.GetMaximum())
        leg.AddEntry(hist,pi_latex[ptype],'f')
        leg.Draw()
        histos.append(hist)
        legends.append(leg)
    if(qty in ['cluster_nCells','clusterE', 'cluster_EM_PROBABILITY', 'cluster_sumCellE']): rt.gPad.SetLogy()
canvas.Draw()
canvas.SaveAs(plotpath+'hist_pi0_plus_minus.pdf')

## TensorFlow/Keras prep

In [9]:
ngpu = 1
gpu_list = ["/gpu:"+str(i) for i in range(ngpu)]
#gpu_list = ["/gpu:0"] #["/gpu:0","/gpu:1","/gpu:2","/gpu:3"]

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # disable some of the tensorflow info printouts, only display errors
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
# strategy = tf.distribute.MirroredStrategy()
ngpu = strategy.num_replicas_in_sync
print ('Number of devices: {}'.format(ngpu))
from util.classification.models import baseline_nn_model, baseline_cnn_model, emb_cnn_model, all_cnn_model, merged_cnn_model, merged_cnn_2p_model, resnet, simple_combine_model
from util.classification.models_exp import exp_cnn_model

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


Let's prepare some callbacks (originally from our regression notebook).

In [10]:
# For storing models and some of their metrics (acc, loss)
models = {}
model_history = {}
model_scores = {}
model_performance = {}

# For storing info on ROC curves
roc_fpr = {}
roc_tpr = {}
roc_thresh = {}
roc_auc = {}

Let's add some info to our dictionaries that corresponds to the existing `EM LC Prob` method.

In [11]:
model_key = 'LC EMProb'
model_scores[model_key] = 1-pdata_merged["cluster_EM_PROBABILITY"]
cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    drawPlots=False
)

Area under curve for LC EMProb: 0.9388561940725229


## Simple feed-forward Neural Network
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>

First, we're going to train simple, feed-foward neural networks -- one per calo layer. These will be our "baseline networks".

Note that while for most of the notebook, we'll train one instance of a network per model, whereas here we will explicitly train multiple instances as we're doing one instance *per calo layer*, each for the same model.

In [ ]:
lr = 5e-5
gamma = 0.1
min_delta = 0.0001
patience = 3
dropout = 0.1 # < 0 -> no dropout
#model = baseline_nn_model(strategy, lr=lr, dropout=dropout)

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts

model_name = 'flat'
model_dir = modelpath + model_name # directory for loading/saving flat models

for layer in layers:
    
    model_key = '{}_{}'.format(model_name, layer)
    modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
    npix = cell_shapes[layer][0] * cell_shapes[layer][1]
    models[model_key] = baseline_nn_model(strategy, npix, lr=lr, dropout=dropout)
    
    # train the network
    models[model_key], model_history[model_key] = ctu.TrainNetwork(
        model=models[model_key], 
        modelfile=modelfile, 
        x_train = pcells_merged[layer][pdata_merged.train], 
        y_train = plabels[pdata_merged.train], 
        x_valid = pcells_merged[layer][pdata_merged.val], 
        y_valid = plabels[pdata_merged.val], 
        callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
        epochs=nepochs, 
        batch_size=batch_size, 
        verbose=verbose, 
        overwriteModel=overwriteModel,
        finishTraining=finishTraining
    )
        
    # get performance metric from test set
    model_performance[model_key] = models[model_key].evaluate(
        pcells_merged[layer][pdata_merged.test],
        plabels[pdata_merged.test],
        verbose=0
    )
    
    print('Finished layer {}.'.format(layer))
    
    # get/recalculate network scores for the dataset
    model_scores[model_key] = models[model_key].predict(pcells_merged[layer])[:,1]

Let's look at accuracy and loss, as well as ROC curves, for each network.

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_flat',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10)
)

## Combination Network
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>

Here we train a simple combination network. Its inputs will be the *outputs* of our simple, feed-forward neural networks from above.

In [ ]:
model_name = 'simple'
model_dir = modelpath + model_name # directory for loading/saving simple combination network
model_key = 'simpleCombine'

model_scores_stack = np.column_stack( [model_scores['flat_{}'.format(layer)] for layer in layers])
n_input = model_scores_stack.shape[1]

lr = 1e-3
gamma = 0.1
min_delta = 0.0005
patience = 5

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key,file_extension)
models[model_key] = simple_combine_model(strategy, lr=lr, n_input=n_input)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = model_scores_stack[pdata_merged.train],
    y_train = plabels[pdata_merged.train], 
    x_valid = model_scores_stack[pdata_merged.val], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    model_scores_stack[pdata_merged.test],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(model_scores_stack)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys = ['simpleCombine'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_simple',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys = ['LC EMProb', 'flat_EMB1', 'simpleCombine']
)

## Convolutional Neural Networks
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
Let's try some convolutional neural networks -- afterwards we'll also try an implementation of ResNet.

In [ ]:
pcells_merged_unflattened = cdu.ReshapeImages(pcells_merged, cell_shapes, use_layer_names=True)
rn_data = cdu.DictionarySplit(
    pcells_merged_unflattened, 
    train_indices=pdata_merged.train,
    validation_indices = pdata_merged.val,
    test_indices = pdata_merged.test
)

### Single-calo-layer CNN's
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
First up, we can try creating a set of CNN's, where each only uses an image from a single calorimeter layer.

In [ ]:
lr = 5e-5
gamma = 0.1
min_delta = 0.0001
patience = 3
augmentation = True
nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts

model_name = 'cnn'
model_dir = modelpath + model_name # directory for loading/saving flat models

filters = {
    'EMB1': (2,4), 
    'EMB2': (4,4), 
    'EMB3': (4,2), 
    'TileBar0': (2,2), 
    'TileBar1': (2,2), 
    'TileBar2': (1,1)
}
pools = {
    'EMB1': (1,1), 
    'EMB2': (2,2), 
    'EMB3': (1,1), 
    'TileBar0': (1,1), 
    'TileBar1': (1,1), 
    'TileBar2': (1,1)
}

for layer in layers:
    model_key = '{}_{}'.format(model_name, layer)
    modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)    
    models[model_key] = baseline_cnn_model(input_shape=cell_shapes[layer], f=filters[layer], pool=pools[layer], lr=lr, augmentation=augmentation)
    
    # train the network
    models[model_key], model_history[model_key] = ctu.TrainNetwork(
        model=models[model_key], 
        modelfile=modelfile, 
        x_train = rn_data['train'][layer], 
        y_train = plabels[pdata_merged.train], 
        x_valid = rn_data['valid'][layer],
        y_valid = plabels[pdata_merged.val], 
        callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
        epochs=nepochs, 
        batch_size=batch_size, 
        verbose=verbose, 
        overwriteModel=overwriteModel,
        finishTraining=finishTraining
    )
        
    # get performance metric from test set
    model_performance[model_key] = models[model_key].evaluate(
        rn_data['test'][layer],
        plabels[pdata_merged.test],
        verbose=0
    )
    
    print('Finished layer {}.'.format(layer))
    
    # get/recalculate network scores for the dataset
    model_scores[model_key] = models[model_key].predict(pcells_merged_unflattened[layer])[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    plotstyle=plotstyle,
    model_keys = ['cnn_{}'.format(layer) for layer in layers]
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_cnn',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys = ['cnn_{}'.format(layer) for layer in layers]
)

### Combination Network: Take 2
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
Let's try another combination network, using our single layer CNN's.

In [ ]:
model_name = 'simple_cnn'
model_dir = modelpath + model_name # directory for loading/saving simple combination network
model_key = 'simpleCombine_cnn'

model_scores_stack = np.column_stack([model_scores['cnn_{}'.format(layer)] for layer in layers])
n_input = model_scores_stack.shape[1]

lr = 1e-3
gamma = 0.1
min_delta = 0.0001
patience = 3

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key,file_extension)
models[model_key] = simple_combine_model(strategy, lr=lr, n_input=n_input)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = model_scores_stack[pdata_merged.train],
    y_train = plabels[pdata_merged.train], 
    x_valid = model_scores_stack[pdata_merged.val], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    model_scores_stack[pdata_merged.test],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(model_scores_stack)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    plotstyle=plotstyle,
    model_keys = ['simpleCombine_cnn']
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_simple_cnn',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys = ['LC EMProb', 'simpleCombine', 'simpleCombine_cnn']
)

## Convolutional Neural Networks: More complicated architectures
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
We can also consider more complex CNN's.

### 3 EMB layers, separate
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
First, we can try one that uses all three EMB layers as input.

In [ ]:
pcells_merged_unflattened = cdu.ReshapeImages(pcells_merged, cell_shapes, use_layer_names=True)
# keep only the EMB layers
pcells_merged_unflattened = {key:pcells_merged_unflattened[key] for key in ['EMB1','EMB2','EMB3']}

rn_data = cdu.DictionarySplit(
    pcells_merged_unflattened, 
    train_indices=pdata_merged.train,
    validation_indices = pdata_merged.val,
    test_indices = pdata_merged.test
)

rn_data = {key: list(val.values()) for key,val in rn_data.items()}

In [ ]:
model_name = 'cnn'
model_dir = modelpath + model_name # directory for loading/saving ResNet
model_key = 'cnn_EMB_all'

lr = 1e-4
gamma = 0.05
min_delta = 0.001
patience = 10
augmentation=True

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
models[model_key] = model = emb_cnn_model(
    lr=lr,
    augmentation=augmentation
)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = rn_data['train'], 
    y_train = plabels[pdata_merged.train], 
    x_valid = rn_data['valid'], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    rn_data['test'],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(pcells_merged_unflattened)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys=['cnn_EMB_all'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_cnn_EMB_all',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys=['LC EMProb', 'simpleCombine_cnn', 'cnn_EMB_all']
)

### All layers, separate
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
Similarly, we can try a network using all six calo layers, as separate 1-channel images.

In [ ]:
pcells_merged_unflattened = cdu.ReshapeImages(pcells_merged, cell_shapes, use_layer_names=True)

rn_data = cdu.DictionarySplit(
    pcells_merged_unflattened, 
    train_indices=pdata_merged.train,
    validation_indices = pdata_merged.val,
    test_indices = pdata_merged.test
)

rn_data = {key: list(val.values()) for key,val in rn_data.items()}

In [ ]:
model_name = 'cnn'
model_dir = modelpath + model_name # directory for loading/saving ResNet
model_key = 'cnn_all'

lr = 1e-4
gamma = 0.05
min_delta = 0.001
patience = 10
augmentation=True

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
models[model_key] = model = all_cnn_model(
    lr=lr,
    augmentation=augmentation
)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = rn_data['train'], 
    y_train = plabels[pdata_merged.train], 
    x_valid = rn_data['valid'], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    rn_data['test'],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(pcells_merged_unflattened)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys=['cnn_all'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_cnn_all',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys=['LC EMProb', 'simpleCombine_cnn', 'cnn_EMB_all', 'cnn_all']
)

### All layers, merged
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
Of course, we can also treat the 6 calo layers as 6 channels of the *same* image. Note that this will require some rescaling of the images so that their dimensions match -- we'll also be careful to preserve their integrals. This last point is probably even more relevant for regression tasks (i.e. predicting energy).

In [ ]:
pcells_merged_unflattened = cdu.ReshapeImages(pcells_merged, cell_shapes, use_layer_names=True)
rn_data = cdu.DictionarySplit(
    pcells_merged_unflattened, 
    train_indices=pdata_merged.train,
    validation_indices = pdata_merged.val,
    test_indices = pdata_merged.test
)
rn_data = {key: list(val.values()) for key,val in rn_data.items()}

In [ ]:
model_name = 'cnn'
model_dir = modelpath + model_name # directory for loading/saving ResNet
model_key = 'cnn_merged'

input_shape = (16,16)
dropout = 0.2

lr = 1e-4
gamma = 0.05
min_delta = 0.001
patience = 10
augmentation=True

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
models[model_key] = model = merged_cnn_model(
    lr=lr,
    input_shape=input_shape,
    dropout=dropout,
    augmentation=augmentation
)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = rn_data['train'], 
    y_train = plabels[pdata_merged.train], 
    x_valid = rn_data['valid'], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    rn_data['test'],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(pcells_merged_unflattened)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys=['cnn_merged'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_cnn_merged',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys=['LC EMProb', 'simpleCombine_cnn', 'cnn_EMB_all', 'cnn_all', 'cnn_merged']
)

### EMB merged + TileBar merged
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
As another variation on this theme, we can try a "merged" CNN where we separately treat the 3 EMB layers and the 3 TileBar layers. This will allow us to perform separate rescalings for each -- and we may thus be able to have fewer parameters.

In [ ]:
pcells_merged_unflattened = cdu.ReshapeImages(pcells_merged, cell_shapes, use_layer_names=True)
rn_data = cdu.DictionarySplit(
    pcells_merged_unflattened, 
    train_indices=pdata_merged.train,
    validation_indices = pdata_merged.val,
    test_indices = pdata_merged.test
)
rn_data = {key: list(val.values()) for key,val in rn_data.items()}

In [ ]:
model_name = 'cnn'
model_dir = modelpath + model_name # directory for loading/saving ResNet
model_key = 'cnn_merged_2p'

input_shape1 = (16,16)
input_shape2 = (4,4)
dropout = 0.2

lr = 1e-4
gamma = 0.05
min_delta = 0.001
patience = 10
augmentation=True

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
models[model_key] = model = merged_cnn_2p_model(
    lr=lr,
    input_shape1=input_shape1,
    input_shape2=input_shape2,
    dropout=dropout,
    augmentation=augmentation
)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = rn_data['train'], 
    y_train = plabels[pdata_merged.train], 
    x_valid = rn_data['valid'], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    rn_data['test'],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(pcells_merged_unflattened)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys=['cnn_merged_2p'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_cnn_merged_2p',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys=['LC EMProb', 'cnn_merged', 'cnn_merged_2p']
)

### EMB merged + TileBar depth
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
As an experiment, we can try a network where we use a CNN approach for the EMB layers, whereas for TileBar we just use the integrals of the 3 Tilebar layers. In other words, we don't fully use TileBar images, but just get some depth info from how populated they are.

In [12]:
pcells_merged_unflattened = cdu.ReshapeImages(pcells_merged, cell_shapes, use_layer_names=True)
rn_data = cdu.DictionarySplit(
    pcells_merged_unflattened, 
    train_indices=pdata_merged.train,
    validation_indices = pdata_merged.val,
    test_indices = pdata_merged.test
)
rn_data = {key: list(val.values()) for key,val in rn_data.items()}

In [13]:
model_name = 'cnn'
model_dir = modelpath + model_name # directory for loading/saving ResNet
model_key = 'cnn_exp'

input_shape = (16,16)
dropout = 0.2

lr = 1e-4
gamma = 0.05
min_delta = 0.001
patience = 10
augmentation=True

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
models[model_key] = model = exp_cnn_model(
    lr=lr,
    input_shape=input_shape,
    dropout=dropout,
    augmentation=augmentation
)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = rn_data['train'], 
    y_train = plabels[pdata_merged.train], 
    x_valid = rn_data['valid'], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    rn_data['test'],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(pcells_merged_unflattened)[:,1]

Epoch 1/100
1483/1483 [==============================] - 11s 7ms/step - loss: 0.4262 - acc: 0.8150 - val_loss: 0.3727 - val_acc: 0.8431 - lr: 1.0000e-04
Epoch 2/100
1483/1483 [==============================] - 11s 7ms/step - loss: 0.3600 - acc: 0.8571 - val_loss: 0.3369 - val_acc: 0.8746 - lr: 9.5123e-05
Epoch 3/100
1483/1483 [==============================] - 10s 7ms/step - loss: 0.3344 - acc: 0.8715 - val_loss: 0.3188 - val_acc: 0.8824 - lr: 9.0484e-05
Epoch 4/100
1483/1483 [==============================] - 10s 7ms/step - loss: 0.3195 - acc: 0.8784 - val_loss: 0.3063 - val_acc: 0.8871 - lr: 8.6071e-05
Epoch 5/100
1483/1483 [==============================] - 10s 7ms/step - loss: 0.3084 - acc: 0.8859 - val_loss: 0.2942 - val_acc: 0.8945 - lr: 8.1873e-05
Epoch 6/100
1483/1483 [==============================] - 10s 7ms/step - loss: 0.2993 - acc: 0.8915 - val_loss: 0.2875 - val_acc: 0.8958 - lr: 7.7880e-05
Epoch 7/100
1483/1483 [==============================] - 10s 7ms/step - loss: 0.29

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys=['cnn_exp'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_cnn_exp',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys=['LC EMProb', 'cnn_merged', 'cnn_merged_2p', 'cnn_exp']
)

## ResNet
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
We can also train an instance of ResNet. As with our multi-layer CNN's, we'll need to perform some up/downscaling of images, so that they all have the same dimensions. As we did for some of our CNN's, we will use the maximum possible granularity, `(128,16)` (corresponding with maximum eta and phi dimensions among all calo layers).

In [ ]:
pcells_merged_unflattened = cdu.ReshapeImages(pcells_merged, cell_shapes, use_layer_names=False)
rn_data = cdu.DictionarySplit(
    pcells_merged_unflattened, 
    train_indices=pdata_merged.train,
    validation_indices = pdata_merged.val,
    test_indices = pdata_merged.test
)

We can run a quick test to make sure that our image scaling is working as expected.

In [ ]:
image_test=False

from util.keras.layers import ImageScaleBlock
def TestImage(idxs):
    images = [np.stack([rn_train['input{}'.format(x)][idx,:,:] for idx in idxs],axis=0) for x in range(6)]    
    # need to add the last dimension, corresponding with channel (will be of size 1)
    images = [np.expand_dims(im, axis=-1) for im in images]
    return images

if(image_test):
    test_idx = np.arange(5)
    image = TestImage(test_idx)
    scaled_image = ImageScaleBlock((128,16),normalization=True)(image).numpy()
    for i,im in enumerate(image):
        integrals_old = np.sum(im,axis=(1,2)).flatten()
        integrals_new = np.sum(scaled_image[:,:,:,i], axis=(1,2)).flatten()

        ratio = integrals_old.copy()
        ratio[ratio==0.] = 1.
        ratio = integrals_new/ratio

        integrals_old = '\t\t'.join(['{:.1e}'.format(x) for x in integrals_old])
        integrals_new = '\t\t'.join(['{:.1e}'.format(x) for x in integrals_new])
        ratio         = '\t\t'.join(['{:.1e}'.format(x) for x in ratio        ])

        print('Integral {} = {}'.format(i,integrals_old))
        print('           = {}'.format(integrals_new))
        print('Ratios     = {}'.format(ratio))
        print()

In [ ]:
#tf.keras.backend.set_image_data_format('channels_last')
model_name = 'resnet'
model_dir = modelpath + model_name # directory for loading/saving ResNet
model_key = 'resnet'

lr = 1e-4
gamma = 0.05
min_delta = 0.001
patience = 20 # large patience since loss sometimes fluctuates upwards for a bit?
input_shape = (128,16)
channels = 6
augmentation=True
normalization=True
filter_sets = [
    [64,64,256],
    [128,128,512]
    #[256,256,1024],
    #[512,512,2048]
]         
f_vals = [3,3] # [3,3,3,3] sizes of filters in middle of conv/identity blocks
s_vals = [1,2] # [1,2,2,2] strides for each convolutional block
i_vals = [2,3] # [2,3,5,2] number of identity blocks per stage

nepochs = 200
batch_size = 50 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
models[model_key] = model = resnet(
    filter_sets=filter_sets, 
    lr=lr, 
    channels=channels, 
    f_vals=f_vals, 
    s_vals=s_vals, 
    i_vals=i_vals, 
    input_shape=input_shape, 
    augmentation=augmentation,
    normalization=normalization
)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = rn_data['train'], 
    y_train = plabels[pdata_merged.train], 
    x_valid = rn_data['valid'], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    rn_data['test'],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(pcells_merged_unflattened)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys=['resnet'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_resnet',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys=['LC EMProb', 'flat_EMB1', 'resnet']
)

## Combination Network: Take 3
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>
For fun, we can also train a combination network where we use our flat DNN's, and ResNet as an additional input.

In [ ]:
model_scores_stack = np.column_stack( [model_scores['flat_{}'.format(layer)] for layer in layers])
model_scores_stack = np.column_stack((model_scores_stack, model_scores['resnet']))
n_input = model_scores_stack.shape[1]

nepochs = 50
batch_size = 200*ngpu
verbose = 2


model_name = 'simple_resnet'
model_dir = modelpath + model_name # directory for loading/saving ResNet
model_key = 'simpleCombine_resnet'

lr = 1e-3
gamma = 0.1
min_delta = 0.001
patience = 5

nepochs = 100
batch_size = 200 * ngpu
verbose = 1 # 2 for a lot of printouts
    
modelfile = '{}/{}{}'.format(model_dir,model_key, file_extension)
models[model_key] = simple_combine_model(strategy, lr=lr, n_input=n_input)
    
# train the network
models[model_key], model_history[model_key] = ctu.TrainNetwork(
    model=models[model_key], 
    modelfile=modelfile, 
    x_train = model_scores_stack[pdata_merged.train],
    y_train = plabels[pdata_merged.train], 
    x_valid = model_scores_stack[pdata_merged.val], 
    y_valid = plabels[pdata_merged.val], 
    callbacks = GetCallbacks(modelfile, append=True, use_decay=True, gamma=gamma, min_delta=min_delta, patience=patience), 
    epochs=nepochs, 
    batch_size=batch_size, 
    verbose=verbose, 
    overwriteModel=overwriteModel,
    finishTraining=finishTraining
)
        
# get performance metric from test set
model_performance[model_key] = models[model_key].evaluate(
    model_scores_stack[pdata_merged.test],
    plabels[pdata_merged.test],
    verbose=0
)
    
# get/recalculate network scores for the dataset
model_scores[model_key] = models[model_key].predict(model_scores_stack)[:,1]

In [ ]:
cpu.MetricPlot(
    model_history,
    plotpath=plotpath,
    model_keys = ['simpleCombine_resnet'],
    plotstyle=plotstyle
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_simple_resnet',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys = ['LC EMProb', 'simpleCombine', 'resnet', 'simpleCombine_cnn', 'simpleCombine_resnet']
)

## Summary Plots
<div style="text-align: right"> <a href="#Quick-Navigation:">Top</a> </div>

After we've trained networks to our hearts' content, we can make a set of ROC curves to summarize our results.

In [ ]:
cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_summary',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys = ['LC EMProb', 'flat_EMB1', 'flat_EMB2', 'flat_EMB3', 'cnn_EMB1', 'cnn_EMB2', 'cnn_EMB3', 'simpleCombine', 'simpleCombine_cnn', 'cnn_EMB_all', 'cnn_all', 'cnn_merged', 'resnet', 'simpleCombine_resnet']
)

cpu.RocCurves(
    model_scores,
    data_labels=plabels[:,1],
    indices=pdata_merged.test, 
    roc_fpr=roc_fpr, 
    roc_tpr=roc_tpr, 
    roc_thresh=roc_thresh, 
    roc_auc=roc_auc, 
    plotpath=plotpath,
    plotname='ROC_best',
    drawPlots=True,
    plotstyle=plotstyle,
    figsize=(30,10),
    model_keys = ['LC EMProb', 'flat_EMB1', 'simpleCombine_cnn', 'simpleCombine', 'cnn_EMB_all', 'cnn_all', 'cnn_merged', 'resnet', 'simpleCombine_resnet']
)

Let's also make a scatterplot of number of learnable parameters versus accuracy. The number of parameters can be thought of as a stand-in for complexity (though I assume that computational complexity/cost will also depend on the exact operations being performed).

Note that there is some inherent hard-coding being done here, since some of our models (e.g. `simpleCombine`) use other models as input -- and for such models, we want to count their inputs' weights plus their own.

In [ ]:
import tensorflow.keras.backend as K

complexity_scatter = {}

for key, model in models.items():
    trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
    accuracy        = model_performance[key][1]
    auc             = roc_auc[key]
    complexity_scatter[key] = [trainable_count, accuracy, auc]
    
# manually adjust complexity_scatter for our combination networks

flat_weights = np.sum([np.sum([K.count_params(w) for w in models[key].trainable_weights]) for key in ['flat_{}'.format(layer) for layer in layers]])
cnn_weights = np.sum([np.sum([K.count_params(w) for w in models[key].trainable_weights]) for key in ['cnn_{}'.format(layer) for layer in layers]])

complexity_scatter['simpleCombine'][0] += flat_weights
complexity_scatter['simpleCombine_cnn'][0] += cnn_weights
complexity_scatter['simpleCombine_resnet'][0] += flat_weights + complexity_scatter['resnet'][0]
    
fig, ax = plt.subplots(1,2, figsize=(25,10))
for axis in ax: plotstyle.SetStylePlt(axis)

# Make a scatter -- there will be a lot of points, so consider not drawing them for each network.
draw_keys = ['flat_EMB1', 'simpleCombine_cnn', 'simpleCombine', 'cnn_EMB_all', 'cnn_all', 'cnn_merged', 'resnet', 'simpleCombine_resnet']
draw_cols = [plotstyle.colors[(i+1) % (len(plotstyle.colors)-1)] for i in range(len(draw_keys))]

for i,key in enumerate(draw_keys):
    if(key not in complexity_scatter.keys()): continue
    x,y1,y2 = complexity_scatter[key]
    ax[0].scatter(x=x,y=y1, label=key, color=draw_cols[i], s=64)
    ax[1].scatter(x=x,y=y2, label=key, color=draw_cols[i], s=64)
    
    ax[0].set_xlabel('# Learnable Parameters')
    ax[1].set_xlabel('# Learnable Parameters')
    
    ax[0].set_ylabel('Accuracy')
    ax[1].set_ylabel('Area under ROC curve')
    
    ax[0].set_xlim([0.,2.5e6])
    ax[1].set_xlim([0.,2.5e6])

legs = []
for i,axis in enumerate(ax):
    legs.append(axis.legend(facecolor=plotstyle.canv_plt))
    for leg_text in legs[-1].get_texts(): leg_text.set_color(plotstyle.text_plt)
        
qu.SaveSubplots(fig, ax, ['acc_params', 'auc_params'], savedir=plotpath, ps=plotstyle)

From these plots, `cnn_all` looks like a relatively appealing option -- it's one of the highest-performing networks, and it actually has the fewest learnable parameters. `resnet` performs slightly better, but at the cost of many more learnable weights. This added complexity may translate to slower deployment time, and cause issues if we try to use this network in a triggering context (i.e. on an FPGA), where we will have to deal with significant hardware and timing constraints.

It would be nice to also make a table (or plot) depicting the false positive rate at different true positive rate (i.e. pick out a few reference points from the ROC curves).

In [ ]:
from scipy.interpolate import interp1d
fpr_data = {}
tpr_vals = np.array([0.6,0.8,0.9,0.95])

for tpr_val in tpr_vals: 
    fpr_data[tpr_val] = {}
    for i, draw_key in enumerate(draw_keys):
        
        # Want to find the fpr at the given tpr
        fpr = roc_fpr[draw_key]
        tpr = roc_tpr[draw_key]
        f = interp1d(tpr, fpr)        
        fpr_data[tpr_val][draw_key] = f(tpr_val)
        

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
plotstyle.SetStylePlt(ax)

# multiple bars -- handle visual offsets
bar_width = 0.2
n = len(tpr_vals)
offsets = np.linspace(0.,(n-1) * bar_width,n)
offsets = offsets - np.mean(offsets)

# hatches for distinguishing between different tpr values
hatches = ['/','\\','//','\\\\']

x = np.arange(len(draw_keys))
rects = []
for i,tpr_val in enumerate(fpr_data.keys()):
    bar = ax.bar(
        x + offsets[i], 
        fpr_data[tpr_val].values(),
        bar_width,
        label='@ True positive rate = {:.2f}'.format(tpr_val), 
        color=draw_cols,
        hatch = hatches[i],
        edgecolor=plotstyle.text_plt
    )
    
    for rect in bar:
        height = rect.get_height()
        ax.annotate('{:.2f}'.format(height),
            xy=(rect.get_x() + rect.get_width() / 2, height),
            xytext=(0, 3),  # 3 points vertical offset
            textcoords="offset points",
            color=plotstyle.text_plt,
            ha='center', va='bottom'
                   )
    
    rects.append(bar)

result = ax.set_xticks(x)
result = ax.set_xticklabels(draw_keys)

ax.set_xlabel('Network')
ax.set_ylabel('False positive rate')

leg = ax.legend(facecolor=plotstyle.canv_plt)
for leg_text in leg.get_texts(): leg_text.set_color(plotstyle.text_plt)
    
qu.SaveSubplots(fig, np.array([ax]), ['fpr'], savedir=plotpath, ps=plotstyle)